# 第11章 符合Python风格的对象

## 11.1 对象字符串表示形式

- repr()：便于开发者理解的方式返回对象的字符串表示形式。支持方法是`__repr__`。
- str()：便于用户理解的方式返回对象的字符串表示形式，支持方法是`__str__`。
- bytes()：获取对象的字节序列表示形式，支持方法是`__bytes__`。
- format()和str.format()：以特殊的格式化代码显示对象的字符串表示形式，支持方法是`obj.__format__(format_spec)`。

## 11.2 向量类实现

**需求：**
1. Vector2d实例的分量可以直接通过属性访问（无需调用读值方法）。
2. Vector2d实例可以拆包成变量元组。
3. Vector2d实例的表示形式模拟源码构建的实例的形式。
4. 可以支持`==`比较。
5. 支持`bytes`函数，输出实例的二进制表示形式。
6. 支持`abs`函数，返回Vector2d实例的模。
7. 支持`bool`函数，如果Vector2d实例的模为零，就返回False，否则返回True。

In [22]:
from array import array
import math


class Vector2d_v1:
    # 在Vector2d实例和字节序列之间转换时使用
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)   
        self.y = float(y)

    def __iter__(self):
        # 将实例变成可迭代对象
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        # 使用!r获取各个分量的表示形式
        return '{}({!r}, {!r})'.format(class_name, *self) 

    def __str__(self):
        # 得到一个元组，显示为有序对
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  
                bytes(array(self.typecode, self)))  

    def __eq__(self, other):
        return tuple(self) == tuple(other)  

    def __abs__(self):
        # 计算x分量和y分量构成的直角三角形的斜边长
        return math.hypot(self.x, self.y)  

    def __bool__(self):
        return bool(abs(self)) 
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            # 如果以p结尾，使用极坐标
            fmt_spec = fmt_spec[:-1]
            # 构建元组表示极坐标
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>' 
        else:
            # 如果不以p结尾，使用x分量和y分量构建直角坐标
            coords = self 
            outer_fmt = '({}, {})'  
        components = (format(c, fmt_spec) for c in coords) 
        return outer_fmt.format(*components)
    
    @classmethod 
    def frombytes(cls, octets):
        # 从第一个字节中读取tpyecode
        typecode = chr(octets[0]) 
        # 创建一个memoryview，使用typecode进行转换
        memv = memoryview(octets[1:]).cast(typecode)
        # 得到构造函数所需的一对参数
        return cls(*memv)
    
        

In [23]:
v1 = Vector2d_v1(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [24]:
x, y = v1
x, y

(3.0, 4.0)

In [25]:
v1

Vector2d_v1(3.0, 4.0)

In [26]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [27]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [28]:
abs(v1)

5.0

In [29]:
bool(v1), bool(Vector2d_v1(0, 0))

(True, False)

## 11.3 classmethod与staticmethod

- classmethod：定义操作类而不是操作实例的方法，常见用途是定义备选构造函数。
- staticmethod：静态方法，不是特别有用。

## 11.4 格式化显示

格式规范微语言为一些内置类型提供了专用的表示代码，`b`和`x`分别表示二进制和十六进制的`int`类型，`f`表示小数形式的`float`类型，`%`表示百分数形式。

In [30]:
format(Vector2d_v1(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [31]:
format(Vector2d_v1(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [32]:
format(Vector2d_v1(1, 1), '0.5fp')

'<1.41421, 0.78540>'

## 11.5 可哈希的Vector2d和支持位置模式匹配

In [33]:
from array import array
import math

class Vector2d_v2:
    __match_args__ = ('x', 'y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [34]:
# 支持可哈希
v1 = Vector2d_v2(3, 4)
v2 = Vector2d_v2(3.1, 4.2)
hash(v1), hash(v2)

(1079245023883434373, 1994163070182233067)

In [35]:
{v1, v2}

{Vector2d_v2(3.0, 4.0), Vector2d_v2(3.1, 4.2)}

## 11.6 Python私有属性和“受保护”的属性

- Python没有私有属性的保护机制，在前面加上两个前导下划线，尾部没有或最多有一个下划线，则会进行**名称改写**。
- 约定使用一个下划线前缀编写“受保护”的属性，Python解释器不会对使用单个下划线的属性名做特殊处理。

## 11.7 使用`__slots__`节省空间

In [39]:
class Vector2d_v3(Vector2d_v2):
    # 位置模式匹配可用的公开属性名称
    __match_args__ = ('x', 'y')
    # 列出实例属性名称
    __slots__ = ('__x', '__y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [40]:
v1 = Vector2d_v3(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [41]:
len(dumpd)

17

In [42]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [43]:
len(dumpf)

9

## 11.8 杂谈

特性有助于减少前期投入：

- 先用公开属性的方式定义类，如果以后需要对读值方法和设值方法增加控制，可以通过特性实现，对一开始通过公开属性的名称与对象交互的代码没有影响。
- Java语言不能这样做，需要前期就编写读值方法和设值方法。